In [1]:
import hail as hl
import pandas as pd
import numpy as np
hl.init()

Running on Apache Spark version 2.4.4
SparkUI available at http://a783b4e25167:4042
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.30-2ae07d872f43
LOGGING: writing to /home/eczech/repos/gwas-analysis/notebooks/organism/canine/hail-20200212-2155-0.2.30-2ae07d872f43.log


### MAF and Call Rates

In [131]:
mt = hl.balding_nichols_model(3, 6, 10)
mt = mt.annotate_entries(GT=hl.case().when(hl.rand_bool(.5, seed=1), mt.GT).or_missing())
mt = hl.variant_qc(mt)
mt.count()

2020-02-12 23:02:12 Hail: INFO: balding_nichols_model: generating genotypes for 3 populations, 6 samples, and 10 variants...


(10, 6)

In [132]:
mt.aggregate_entries(hl.agg.fraction(hl.is_defined(mt.GT)))

0.48333333333333334

In [133]:
m = hl.linalg.BlockMatrix.from_entry_expr(
    hl.case().when(hl.is_defined(mt.GT), mt.GT.n_alt_alleles()).default(-1)
).to_numpy()
pd.DataFrame(m, index=mt.locus.collect(), columns=mt.pop.collect())

2020-02-12 23:02:14 Hail: INFO: Coerced sorted dataset
2020-02-12 23:02:14 Hail: INFO: Wrote all 1 blocks of 10 x 6 matrix with block size 4096.


,2,1,2,2,0,0
1:1,1.0,-1.0,2.0,1.0,0.0,2.0
1:2,2.0,2.0,2.0,-1.0,2.0,-1.0
1:3,1.0,-1.0,-1.0,-1.0,2.0,2.0
1:4,-1.0,2.0,-1.0,-1.0,2.0,1.0
1:5,0.0,-1.0,1.0,-1.0,-1.0,-1.0
1:6,2.0,-1.0,1.0,-1.0,-1.0,-1.0
1:7,0.0,-1.0,-1.0,0.0,2.0,-1.0
1:8,2.0,2.0,-1.0,-1.0,2.0,2.0
1:9,-1.0,2.0,-1.0,1.0,-1.0,-1.0
1:10,-1.0,0.0,-1.0,-1.0,-1.0,-1.0


In [134]:
def get_maf(mt):
    mt = mt.annotate_rows(maf=hl.agg.group_by(
        mt.pop, hl.agg.sum(mt.GT.n_alt_alleles()) / (hl.agg.count_where(hl.is_defined(mt.GT)) * 2)))
    return mt

def get_cr(mt):
    mt = mt.annotate_rows(cr=hl.agg.group_by(
        mt.pop, hl.agg.fraction(hl.is_defined(mt.GT))))
    return mt

In [135]:
mt = get_maf(mt)
mt = get_cr(mt)

In [136]:
mt.maf.collect()

2020-02-12 23:02:15 Hail: INFO: Coerced sorted dataset


[{0: 0.5, 1: nan, 2: 0.6666666865348816},
 {0: 1.0, 1: 1.0, 2: 1.0},
 {0: 1.0, 1: nan, 2: 0.5},
 {0: 0.75, 1: 1.0, 2: nan},
 {0: nan, 1: nan, 2: 0.25},
 {0: nan, 1: nan, 2: 0.75},
 {0: 1.0, 1: nan, 2: 0.0},
 {0: 1.0, 1: 1.0, 2: 1.0},
 {0: nan, 1: 1.0, 2: 0.5},
 {0: nan, 1: 0.0, 2: nan}]

In [137]:
pd.Series(pd.DataFrame(mt.maf.collect()).values.ravel()).describe()

2020-02-12 23:02:16 Hail: INFO: Coerced sorted dataset


count    19.000000
mean      0.732456
std       0.348696
min       0.000000
25%       0.500000
50%       1.000000
75%       1.000000
max       1.000000
dtype: float64

In [138]:
mt.aggregate_entries(hl.agg.sum(hl.is_defined(mt.GT)))

29

In [141]:
len(pd.DataFrame(mt.GT.collect()).dropna())

2020-02-12 23:02:33 Hail: INFO: Coerced sorted dataset


29